In [ ]:
import numpy as np
import pystan
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt
%matplotlib inline

# Exercise 2: Rats

Thirty rats have their weight measured once per week shortly after being born, on days 8, 15, 22, 29, and 36. For this dataset, we are going to consider a prediction task: 

**Given the measurements on days 8, 15, and 22, can we predict the rat weight on days 29 and 36?**

As usual, the first thing to do is look at the data! We will subsample it here and plot just a few rows.

In [ ]:
try:
    data = pd.read_csv("rats.csv");
except:
    data = pd.read_csv("https://github.com/tbrx/intro-stan-python/raw/main/rats.csv");
print("Number of rows:", len(data))
data.head()

In [ ]:
past_rats = data[['day8', 'day15', 'day22']]
future_rats = data[['day29', 'day36']]

# plot
f = plt.plot(data[:10].T, 'o--', c='C1');
p = plt.plot(past_rats[:10].T, 'o-', c='C0');
plt.legend([p[0], f[0]], ['Observed data', 'Future data']);
plt.ylabel("Weight");
plt.title("Ten Rats");

### Basic linear regression

The following Stan model implements a basic linear regression model: it estimates two coefficients, $\alpha$ and $\beta$, where $\alpha$ is the average weight of a rat at birth and $\beta$ is the average amount it grows in a week.

This model can be written as

$$\begin{align*}
y_{nt} \sim \mathcal{N}(\alpha + \beta x_t, \sigma_y^2)
\end{align*}$$

where $\sigma_y$ is observation noise, $x_t$ is the age of the rat in days at time $t$, and $y_{nt}$ is the weight of rat $n$ at time $t$.

Take note of both `for` loops used in the `model` and `generated quantities` blocks.

In [ ]:
rats_model_code = """
data {
    int N; // number of rats    
    int T; // number of observed timesteps
    int x[T]; // date of measurement
    real y[N,T]; // matrix of weights by date

    int T_test; // number of held-out timesteps
    int x_test[T_test]; // dates of held-out timesteps
}
parameters {
    real alpha;
    real beta;
    real<lower=0> sigma_y;
}
model {
    sigma_y ~ normal(0, 100); // prior on scale
    for (n in 1:N) {
        for (t in 1:T) {
            y[n,t] ~ normal(alpha + beta * x[t], sigma_y);
        }
    }
}
generated quantities {
    real y_hat[T_test];  // predictions into the future (mean)
    real y_pred[T_test]; // predictions into the future (sampled)
    
    for (t in 1:T_test) {
        y_hat[t] = alpha + beta * x_test[t]; // predictive mean
        y_pred[t] = normal_rng(y_hat[t], sigma_y); // sampled prediction
    }
}
"""
rats_model_basic = pystan.StanModel(model_code=rats_model_code)

In [ ]:
rats_data = {'N': len(past_rats),
             'T': 3,
             'x': [8, 15, 22],
             'y': past_rats,
             'T_test': 2,
             'x_test': [29, 36]};

In [ ]:
fit = rats_model_basic.sampling(data=rats_data, chains=4, iter=2000);
print(fit.stansummary(pars=['alpha', 'beta', 'sigma_y']));

## Task: Model differences between rats

The predictions look fairly good. However, these predictions are the same *for every rat*. It doesn't account for the individual variation in rats at all.

Looking at the plot, we see that some rats are consistently heavier than others, or may grow at consistently faster or slower rates.

In [ ]:
f = plt.plot(data.T, 'o--', c='C1');
p = plt.plot(past_rats.T, 'o-', c='C0');
plt.boxplot(fit['y_pred'], positions=(3, 4));
plt.xticks(np.arange(5), data.columns);

* **Q** How can we modify the model to capture differences in rats?

Think about how to modify the model, and write it here. In order to be compatible with the plotting scripts below, your model should have vectors of length $N$ for `alpha[N]` and `beta[N]`.

In [ ]:
rats_model_code = """
data {
    int N; // number of rats    
    int T; // number of observed timesteps
    real x[T]; // date of measurement
    real y[N,T]; // matrix of weights by date
    
    int T_test; // number of held-out timesteps
    real x_test[T_test]; // dates of held-out timesteps
}
parameters {
    real alpha[N];
    real beta[N];
    real<lower=0> sigma_y;
    
    // 
    //  YOUR CODE HERE
    //
}
model {
    // 
    //  YOUR CODE HERE
    //
}
generated quantities {
    real y_hat[N,T_test]; 
    real y_pred[N,T_test];
    
    for (n in 1:N) {
        for (t in 1:T_test) {
            y_hat[n,t] = alpha[n] + beta[n] * x_test[t]; // predictive mean
            y_pred[n,t] = normal_rng(y_hat[n,t], sigma_y); // sampled prediction
        }
    }
}
"""
rats_model = pystan.StanModel(model_code=rats_model_code)

In [ ]:
fit = rats_model.sampling(data=rats_data, chains=4, iter=2000);
print(fit)

### Plot predictions for each rat

In [ ]:
plt.figure(figsize=(14, 14));
for i in range(len(data)):
    plt.subplot(6, 5, i+1);
    plt.plot(data.loc[i], 'o--', c='C1');
    plt.plot(past_rats.loc[i], 'o-', c='C0');
    plt.boxplot(fit['y_pred'][:,i], positions=(3, 4));
    plt.xticks(np.arange(5), data.columns);
    plt.title('Rat %d: $\hat \\alpha = %0.1f, \hat \\beta = %0.1f$' % (i+1, fit['alpha'][:,i].mean(), fit['beta'][:,i].mean()))
    plt.ylim(110, 410);

plt.tight_layout();

## Task: Predicting weights for new rats

In the previous model, we supposed that for all 30 rats we only observed their weights at days 8, 15, and 22.

Sometimes we might have settings where we have collected complete data for a handful of rats, and are interested in making predictions on new rats.

Here, let's suppose a first experiment measured rats 1-25 for 36 days, and now in a second experiment we are measuring rats 26-30, currently just finishing day 22.

* **Q** What can we learn from having already collected data on 25 rats? Do we expect predictions for rats 26-30 to be the same as in the previous model?


In [ ]:
new_rats_code = """
data {
    int N_obs; // number of fully observed rats
    int N_new; // number of new rats
    
    int T_obs; // number of timestamps for observed rats
    int T_new; // number of timestamps for new rats
    int T_test; // number of timestamps to predict for new rats
    
    int x_obs[T_obs]; // date of measurement
    real y_obs[N_obs,T_obs]; // matrix of weights by date

    int x_new[T_new]; // date of measurement
    real y_new[N_new,T_new]; // matrix of weights by date
    
    int x_test[T_test]; // dates of timestamps to predict
}
parameters {
    real alpha[N_obs + N_new]; // Note: place the observed entries first
    real beta[N_obs + N_new];
    real<lower=0> sigma_y;
    
    // 
    //  YOUR CODE HERE
    //
}
model {
    // 
    //  YOUR CODE HERE
    //
}
generated quantities {
    // just compute predictions for the new rats
    real y_hat[N_new,T_test]; 
    real y_pred[N_new,T_test];
    
    for (n in 1:N_new) {
        for (t in 1:(T_test)) {
            y_hat[n,t] = alpha[N_obs+n] + beta[N_obs+n] * x_test[t]; // predictive mean
            y_pred[n,t] = normal_rng(y_hat[n,t], sigma_y); // sampled prediction
        }
    }
}
"""
new_rats_model = pystan.StanModel(model_code=new_rats_code)

In [ ]:
initial_rats = data[:25]
new_past_rats = data[25:][['day8', 'day15', 'day22']]

split_rats_data = {'N_obs': len(initial_rats),
                   'N_new': len(new_past_rats),
                   'T_obs': 5,
                   'T_new': 3,
                   'T_test': 2,
                   'x_obs': [8, 15, 22, 29, 36],
                   'x_new': [8, 15, 22],
                   'x_test': [29, 36],
                   'y_obs': initial_rats,
                   'y_new': new_past_rats };


In [ ]:
pred_fit = new_rats_model.sampling(data=split_rats_data, chains=4, iter=2000);
print(pred_fit.stansummary())

### Plot predictions on the five new rats

In [ ]:
plt.figure(figsize=(14, 4));
for i in range(5):
    plt.subplot(1, 5, i+1);
    plt.plot(data.loc[25+i], 'o--', c='C1');
    plt.plot(new_past_rats.loc[25+i], 'o-', c='C0');
    plt.boxplot(pred_fit['y_pred'][:,i], positions=(3, 4));
    plt.xticks(np.arange(5), data.columns);
    plt.title('Rat %d: $\hat \\alpha = %0.1f, \hat \\beta = %0.1f$' % (i+26, pred_fit['alpha'][:,25+i].mean(), pred_fit['beta'][:,25+i].mean()))
    plt.ylim(110, 410);

plt.tight_layout();

In [ ]:
plt.figure(figsize=(11,5));
m0 = plt.violinplot(data[25:][['day29', 'day36']].to_numpy().T[:1,:], positions=np.arange(25,30), widths=1.0);
m1 = plt.violinplot(pred_fit['y_pred'][:,:,0], positions=np.arange(25,30)-0.15, widths=0.3);
m2 = plt.violinplot(fit['y_pred'][:,25:,0], positions=np.arange(25,30)+0.15, widths=0.3);
plt.title("Prediction on Day 29")
plt.xlabel("Test Rat")
plt.legend([m0['bodies'][0], m1['bodies'][0], m2['bodies'][0]], ["Actual data", "Updated model", "Original model"]);
plt.ylabel("Weight");

In [ ]:
plt.figure(figsize=(11,5));
m0 = plt.violinplot(data[25:][['day29', 'day36']].to_numpy().T[1:,:], positions=np.arange(25,30), widths=1.0);
m1 = plt.violinplot(pred_fit['y_pred'][:,:,1], positions=np.arange(25,30)-0.15, widths=0.3);
m2 = plt.violinplot(fit['y_pred'][:,25:,1], positions=np.arange(25,30)+0.15, widths=0.3);
plt.title("Prediction on Day 36")
plt.xlabel("Test Rat")
plt.legend([m0['bodies'][0], m1['bodies'][0], m2['bodies'][0]], ["Actual data", "Updated model", "Original model"]);
plt.ylabel("Weight");

* **Q** How do the predictions change? Do these results make sense? 
* **Q** Is this a better fit on these last five rats than the original model, which never saw any measurements at days 29 or 36? Why or why not?
* **Q** What does this imply, if anything, about our choice of linear model.

## Task: Improving the predictive model

Plot the alphas and betas for both these models, for the last five rats. What can we learn from these?

One possibility is that the assumption of our model, that rat weight increases linearly over time, is unrealistic. An advantage of probabilistic programming is it makes it easy to iterate on modeling choices. Try replacing the predictive mean with another function (possibly nonlinear) and see if you can make an improvement. 
